In [ ]:
!pip install pymatgen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 32.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
     

In [ ]:
def read_xyz_md(name):
    """
    Read xyz file:
        first line number of atoms
        second line: first 9 column is 3 unit cell vector, the last column is energy
    :param name:
    :return: a list of cell vectors, coords, atoms, and energies
    """
    f = open(name)

    i = 1  #line index
    m = 0  #frame index

    cells = []
    coords = []
    atoms = []
    energies = []

    atom = []
    cell = []
    coord = []

    nAtom = int(f.readline().split()[0])
    for line in f.readlines():
        line = line.split()

        if (m + 1 < i) and (i < m + nAtom + 2):
            atom.append(line[0])
            coord.append([float(line[1]), float(line[2]), float(line[3])])

        if i == m + 1:
            cell = np.array([float(x) for x in line[:-1]]).reshape(3, 3)
            if len(line) == 10:
                energies.append(float(line[-1]))
            else:
                energies.append(0.0)

        if len(line) == 1 and line[0] == str(nAtom):  # start new frame
            if len(cell) > 1:
                cells.append(cell)
                coords.append(coord)
                atoms.append(atom)
            atom = []
            cell = []
            coord = []
            m = i  # increase one frame
        i += 1
    f.close()
    cells.append(cell)
    coords.append(coord)
    atoms.append(atom)
    return cells, coords, atoms, energies

In [ ]:
from pymatgen.core  import Structure, Element
from sklearn.model_selection import train_test_split
from multiprocessing import Pool
import numpy as np
from sklearn.linear_model import Ridge, LinearRegression
from matplotlib import pyplot as plt
import tensorflow
import tensorflow.keras.backend as K
import numpy as np
from sklearn.model_selection import train_test_split
from multiprocessing import Pool
# from pymatgen import Structure
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Add, Activation, Dropout, BatchNormalization, Conv1D, Conv2D, Concatenate, Dense, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Lambda
from tensorflow.keras.utils import Sequence
from matplotlib import pyplot as plt
import pymatgen as pm
from tensorflow.keras.layers import Multiply
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
# from read_xyz import read_xyz_md

cells, coords, atoms, energies = read_xyz_md('/content/drive/MyDrive/Colab Notebooks/Pairwise_data/2000K_rand.xyz')
X = np.load('/content/drive/MyDrive/Colab Notebooks/Pairwise_data/X.npy', allow_pickle = True)

energies = np.array(energies)
m = energies.mean()
Y = (energies - m) * (27.2116 * 0.5)
X = np.array(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

class Generator(Sequence):
    """Pairs for compare images
    This will create batch_size examples of positive examples and that of negative examples
    """
    def __init__(self, X, y, batch_size = 32, shuffle = True):
        """
        :param X: list of images
        :parm y: labels of images
        :param batch_size:
        :param shuffle:
        """
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(X.shape[0])
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        """
        Generate one batch of data
        """
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size: (index + 1) * self.batch_size]
        X_train = self.X[indexes, :, :, :]
        y_train = self.y[indexes]
        return X_train, y_train

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            np.random.shuffle(self.indexes)





In [ ]:
X.shape

(2500, 64, 120, 2)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(1250, 64, 120, 2)
(1250,)
(1250, 64, 120, 2)
(1250,)


In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
train_generator = Generator(X = X_train, y = y_train, batch_size = 16)
test_generator = Generator(X = X_test, y = y_test, batch_size = 16)

def atomic_feature(x):
    return K.sum(x, axis = -2)

def energy_pool(x):
    return K.sum(x, axis = 1)

INPUT_SHAPE = [64, 120, 2]
P_DROPOUT = 0
L2 = 0.0001
inp = Input(shape = INPUT_SHAPE)


x = Conv2D(filters = 128,
           kernel_size = (1, 1),
           padding = 'same',
           kernel_regularizer = regularizers.l2(L2))(inp)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(P_DROPOUT)(x)

x = Conv2D(filters = 128,
           kernel_size = (1, 1),
           padding = 'same',
           kernel_regularizer = regularizers.l2(L2))(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(P_DROPOUT)(x)

x = Conv2D(filters = 128,
           kernel_size = (1, 1),
           padding = 'same',
           kernel_regularizer = regularizers.l2(L2))(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(P_DROPOUT)(x)

x = atomic_feature(x)

x = Conv1D(filters = 128,
           kernel_size = 1,
           padding = 'same',
           kernel_regularizer = regularizers.l2(L2))(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(P_DROPOUT)(x)

x = Conv1D(filters = 128,
           kernel_size = 1,
           padding = 'same',
           kernel_regularizer = regularizers.l2(L2))(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(P_DROPOUT)(x)

x = Conv1D(filters = 128,
           kernel_size = 1,
           padding = 'same',
           kernel_regularizer = regularizers.l2(L2))(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(P_DROPOUT)(x)

x = Conv1D(filters = 128,
           kernel_size = 1,
           padding = 'same',
           kernel_regularizer = regularizers.l2(L2))(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(P_DROPOUT)(x)

x = Conv1D(filters = 128,
           kernel_size = 1,
           padding = 'same',
           kernel_regularizer = regularizers.l2(L2))(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(P_DROPOUT)(x)

x = Conv1D(filters = 1,
           kernel_size = 1,
           padding = 'same',
           kernel_regularizer = regularizers.l2(L2))(x)
#x = BatchNormalization()(x)
x = Activation('linear')(x)
x = Dropout(P_DROPOUT)(x)

y = energy_pool(x)

model = Model(inputs = inp, outputs = y)
optimizer = tensorflow.keras.optimizers.Adam(learning_rate = 0.00005)
model.compile(optimizer = optimizer,
              loss = 'mse',
              metrics = ['mse', rmse, 'mae'])

model.fit_generator(generator = train_generator,
                    validation_data = test_generator, epochs = 2500)

In [ ]:
import matplotlib.pyplot as plt

loss = model.history['loss']
val_loss = model.history['val_loss']

plt.figure()
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Model Loss vs. Epoch', fontweight= 'bold')
plt.xlabel('Epoch', fontweight= 'bold')
plt.ylabel('Loss', fontweight= 'bold')
plt.legend()
plt.savefig("loss-epoch-energy.png")
plt.show()

In [ ]:
def plot(model, X_test, y_test, name):
  y_pred_test = model.predict(X_test)
  plt.scatter(y_test, y_pred_test, s = 1)
  plt.xlabel('DFT forces', fontweight= 'bold')
  plt.ylabel('Predeicted forces', fontweight= 'bold')
  plt.title('Test set', fontweight= 'bold')

  plt.tight_layout()
  plt.savefig(f"{name}.png")
  plt.show()

In [ ]:
plot(model, X_test, y_test, 'energy')